In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np

import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [2]:
events = pd.read_csv('data/events_up_to_01062018.csv', low_memory = False)
frecuencia = events[['person']]
frecuencia['cant_eventos'] = 1
frecuencia = pd.DataFrame({'cant_eventos': frecuencia.groupby('person').aggregate(sum)['cant_eventos']}).reset_index()
events = events.merge(frecuencia, on = 'person', how = 'inner')
events.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,2018-05-18 00:30:30,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,2018-05-18 00:07:23,search engine hit,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
3,2018-05-18 00:11:56,checkout,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
4,2018-05-18 00:11:35,viewed product,4886f805,NaN,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9


In [3]:
del frecuencia

In [4]:
# prediction_set.person.describe()

In [5]:
# training_set.person.describe()

In [6]:
sets = events
cols = ['event','fecha','person', 'sku', 'model', 'condition', 'storage', 'color','cant_eventos']
cols_visit = ['event','fecha','person','new_vs_returning','city','region','country','device_type','screen_resolution','operating_system_version','browser_version','cant_eventos','url', 'skus', 'search_term', 'staticpage', 'campaign_source', 'search_engine', 'channel']
# cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x']
# rename = {'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color','cant_eventos_x': 'cant_eventos', 'compras_prod_x': 'compras_prod', 'vistas_prod_x': 'vistas_prod' ,'otros_x': 'otros'}
cols_x = ['fecha_x','person','sku_x', 'model_x', 'condition_x', 'storage_x', 'color_x','cant_eventos_x','compras_prod_x','vistas_prod_x', 'otros_x']
rename = {'event_x': 'event', 'fecha_x': 'fecha', 'model_x': 'model','sku_x':'sku', 'condition_x': 'condition', 'storage_x': 'storage', 'color_x': 'color','cant_eventos_x': 'cant_eventos'}

In [7]:
del events

# Set Datos

# #Analisis TP1

In [8]:
sets.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'cant_eventos'],
      dtype='object')

In [9]:
#Cambiamos el tipo de dato de fecha de timestamp a datetime para un mejor analisis y renombramos a "fecha"
sets[['timestamp']] = sets[['timestamp']].apply(pd.to_datetime)
sets.rename({
    'timestamp' : 'fecha',
}, axis=1, inplace=True)
# sets.head()

vamos a crear las columnas compras_prod, vistas_prod, otros para productos y visitas_sitio para persona, de la forma de One Hot Encoder

In [10]:
#Detalles de Productos que fueron comprados
productos = sets.loc[:, cols]
productos_comprados = productos[(productos['event'] == 'conversion') | (productos['event'] == 'checkout')]
productos_comprados.drop(['event'], inplace=True, axis=1)
productos_comprados['compras_prod'] = 1
productos_comprados['vistas_prod'] = 0
frecuencia_compras = productos_comprados[['person']]
frecuencia_compras['cant_compras'] = 1
frecuencia_compras = pd.DataFrame({'cant_compras': frecuencia_compras.groupby('person').aggregate(sum)['cant_compras']}).reset_index()
productos_comprados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod
3,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,1,0
45,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,1,0
121,2018-05-22 20:29:35,0297fc1e,8992.0,iPhone 6S,Bom,32GB,Ouro Rosa,567,1,0
154,2018-05-10 00:39:51,0297fc1e,7029.0,iPhone 6S Plus,Bom,16GB,Cinza espacial,567,1,0
378,2018-02-25 18:38:00,0297fc1e,1342.0,iPhone 6,Muito Bom,64GB,Dourado,567,1,0


In [11]:
productos_comprados.count()

fecha           72406
person          72406
sku             72406
model           72406
condition       72406
storage         72406
color           72406
cant_eventos    72406
compras_prod    72406
vistas_prod     72406
dtype: int64

In [12]:
frecuencia_compras.head()

,person,cant_compras
0,0008ed71,3
1,00091926,2
2,000ba417,7
3,000c79fe,1
4,000e4d9e,1


In [13]:
# productos_comprados = productos_comprados.drop_duplicates()
# productos_comprados.count()

In [14]:
#Detalles de Productos que fueron visitados
productos_visitados = productos.loc[productos.event == 'viewed product']
productos_visitados.drop(['event'], inplace=True, axis=1)
productos_visitados['compras_prod'] = 0
productos_visitados['vistas_prod'] = 1
frecuencia_vistas = productos_visitados[['person']]
frecuencia_vistas['cant_vistas'] = 1
frecuencia_vistas = pd.DataFrame({'cant_vistas': frecuencia_vistas.groupby('person').aggregate(sum)['cant_vistas']}).reset_index()
productos_visitados.head()

,fecha,person,sku,model,condition,storage,color,cant_eventos,compras_prod,vistas_prod
0,2018-05-18 00:11:59,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1
1,2018-05-18 00:30:30,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1
4,2018-05-18 00:11:35,4886f805,9287.0,Samsung Galaxy J7 Prime,Muito Bom,32GB,Dourado,9,0,1
6,2018-05-18 00:11:53,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9,0,1
9,2018-05-18 00:11:27,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65,0,1


In [15]:
productos_visitados.count()

fecha           1248124
person          1248124
sku             1248124
model           1248124
condition       1248124
storage         1248124
color           1248124
cant_eventos    1248124
compras_prod    1248124
vistas_prod     1248124
dtype: int64

In [16]:
frecuencia_vistas.head()

,person,cant_vistas
0,00091926,372
1,00091a7a,3
2,000ba417,153
3,000c79fe,3
4,000e4d9e,339


In [17]:
# productos_visitados = productos_comprados.drop_duplicates()
# productos_visitados.count()

In [18]:
# Eliminamos de productos visitados las personas que estan en productos comprados
# productos_visitados = productos_visitados.merge(productos_comprados, on='person', how='left')
# productos_visitados = productos_visitados[productos_visitados['fecha_y'].isnull()]
# productos_visitados = productos_visitados.loc[:,cols_x]
# productos_visitados.rename(columns=rename, inplace=True)
# productos_visitados.head()

In [19]:
otros = sets.loc[:, cols_visit]
otros = otros[(otros['event'] != 'conversion') & (otros['event'] != 'checkout') & (otros['event'] != 'viewed product') & (otros['event'] != 'visited site')]
otros.drop(['event'], inplace=True, axis=1)
otros['visitas_sitio'] = 0
otros['otros'] = 1
frecuencia_otros = otros[['person']]
frecuencia_otros['cant_otros'] = 1
frecuencia_otros = pd.DataFrame({'cant_otros': frecuencia_otros.groupby('person').aggregate(sum)['cant_otros']}).reset_index()
otros.head()

,fecha,person,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,url,skus,search_term,staticpage,campaign_source,search_engine,channel,visitas_sitio,otros
2,2018-05-18 00:07:23,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,Google,NaN,0,1
5,2018-05-18 00:10:52,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,"11947,9272,9286,10840,10868,10854,6413,6399,6385",J7,NaN,NaN,NaN,NaN,0,1
7,2018-05-18 00:07:22,4886f805,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,"6594,6650,6664,7238,1059,6706,6721,480,12606,7...",NaN,NaN,NaN,NaN,NaN,0,1
13,2018-05-18 00:11:26,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,/comprar/iphone/iphone-5s,NaN,NaN,NaN,criteo,NaN,NaN,0,1
15,2018-05-16 02:48:13,ad93850f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65,/comprar/iphone/iphone-5s,NaN,NaN,NaN,criteo,NaN,NaN,0,1


In [20]:
otros.count()

fecha                       817082
person                      817082
new_vs_returning                 0
city                             0
region                           0
country                          0
device_type                      0
screen_resolution                0
operating_system_version         0
browser_version                  0
cant_eventos                817082
url                         191131
skus                        483810
search_term                 113763
staticpage                   11201
campaign_source             191286
search_engine               106406
channel                          0
visitas_sitio               817082
otros                       817082
dtype: int64

In [21]:
frecuencia_otros.head()

,person,cant_otros
0,0008ed71,1
1,00091926,40
2,00091a7a,6
3,000ba417,40
4,000c79fe,12


In [22]:
# otros = otros.drop_duplicates()
# otros.count()

In [23]:
# Eliminamos de otros las personas que estan en productos comprados
# otros = otros.merge(productos_comprados, on='person', how='left')
# otros = otros[otros['fecha_y'].isnull()]
# otros = otros.loc[:,cols_x]
# otros.rename(columns=rename, inplace=True)
# otros.head()

In [24]:
# Eliminamos de otros las personas que estan en productos visitados
# otros = otros.merge(productos_visitados, on='person', how='left')
# otros = otros[otros['fecha_y'].isnull()]
# otros = otros.loc[:,cols_x]
# otros.rename(columns=rename, inplace=True)
# otros.head()

In [25]:
#Caracterisitcas del dispositivos con el que el usuario visito el sitio
visitas = sets.loc[:,cols_visit]
features_usuario = visitas.loc[visitas.event == 'visited site']
features_usuario.drop(['event'], inplace=True, axis=1)
features_usuario['visitas_sitio'] = 1
features_usuario['otros'] = 0
frecuencia_visitas = features_usuario[['person']]
frecuencia_visitas['cant_visitas_sitio'] = 1
frecuencia_visitas = pd.DataFrame({'cant_visitas_sitio': frecuencia_visitas.groupby('person').aggregate(sum)['cant_visitas_sitio']}).reset_index()
features_usuario.head()

,fecha,person,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,url,skus,search_term,staticpage,campaign_source,search_engine,channel,visitas_sitio,otros
8,2018-05-18 00:07:22,4886f805,New,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1,0
69,2018-05-14 23:50:22,ad93850f,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1,0
70,2018-05-16 02:48:13,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0
71,2018-05-18 00:11:26,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0
72,2018-05-18 22:11:46,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0


In [26]:
features_usuario.count()

fecha                       204069
person                      204069
new_vs_returning            204069
city                        204069
region                      204069
country                     204069
device_type                 204069
screen_resolution           204066
operating_system_version    204069
browser_version             204069
cant_eventos                204069
url                              0
skus                             0
search_term                      0
staticpage                       0
campaign_source                  0
search_engine                    0
channel                     204069
visitas_sitio               204069
otros                       204069
dtype: int64

In [27]:
del productos
del visitas

In [28]:
#concateno usuario visitas sitios y otros
frame = [features_usuario, otros]
features_usuario = pd.concat(frame)
features_usuario.head()

,fecha,person,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,cant_eventos,url,skus,search_term,staticpage,campaign_source,search_engine,channel,visitas_sitio,otros
8,2018-05-18 00:07:22,4886f805,New,Cantagalo,Rio de Janeiro,Brazil,Smartphone,360x640,Android 7,Chrome Mobile 64.0,9,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1,0
69,2018-05-14 23:50:22,ad93850f,New,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1,0
70,2018-05-16 02:48:13,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0
71,2018-05-18 00:11:26,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0
72,2018-05-18 22:11:46,ad93850f,Returning,São Paulo,Sao Paulo,Brazil,Smartphone,360x640,Android 5.1.1,Chrome Mobile 66.0,65,NaN,NaN,NaN,NaN,NaN,NaN,Paid,1,0


In [29]:
del frame
del otros

In [30]:
user_prod_vistas = productos_visitados.merge(features_usuario, on="person", how="left")
user_prod_vistas.head()

MemoryError: 

In [ ]:
user_prod_vistas.count()

In [30]:
user_prod_comprados = productos_comprados.merge(features_usuario, on="person", how="outer")
user_prod_comprados.head()

,fecha_x,person,sku,model,condition,storage,color,cant_eventos_x,compras_prod,vistas_prod,...,cant_eventos_y,url,skus,search_term,staticpage,campaign_source,search_engine,channel,visitas_sitio,otros
0,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,0.0,...,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1.0,0.0
1,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,0.0,...,9.0,NaN,NaN,NaN,NaN,NaN,Google,NaN,0.0,1.0
2,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,0.0,...,9.0,NaN,"11947,9272,9286,10840,10868,10854,6413,6399,6385",J7,NaN,NaN,NaN,NaN,0.0,1.0
3,2018-05-18 00:11:56,4886f805,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,9.0,1.0,0.0,...,9.0,NaN,"6594,6650,6664,7238,1059,6706,6721,480,12606,7...",NaN,NaN,NaN,NaN,NaN,0.0,1.0
4,2018-05-14 23:54:19,ad93850f,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,65.0,1.0,0.0,...,65.0,NaN,NaN,NaN,NaN,NaN,NaN,Organic,1.0,0.0


In [31]:
user_prod_comprados.count()

fecha_x                     5323468
person                      5425278
sku                         5323468
model                       5323468
condition                   5323468
storage                     5323468
color                       5323468
cant_eventos_x              5323468
compras_prod                5323468
vistas_prod                 5323468
fecha_y                     5424793
new_vs_returning            1072142
city                        1072142
region                      1072142
country                     1072142
device_type                 1072142
screen_resolution           1072128
operating_system_version    1072142
browser_version             1072142
cant_eventos_y              5424793
url                          786834
skus                        2888199
search_term                  567764
staticpage                    91388
campaign_source              787781
search_engine                424561
channel                     1072142
visitas_sitio               

In [ ]:
del features_usuario
del productos_comprados
del productos_visitados

In [ ]:
#concateno
frames = [user_prod_comprados, user_prod_vistas]
sets = pd.concat(frames)
sets.head()

In [ ]:
del frames
del user_prod_comprados
del user_prod_vistas

In [ ]:
sets.count()

In [ ]:
# Agregamos las frecuencias de cada persona de compras, vistas, visitas sitio y otros
sets = sets.merge(frecuencia_compras, on = 'person', how = 'left')
sets = sets.merge(frecuencia_vistas, on = 'person', how = 'left')
sets = sets.merge(frecuencia_otros, on = 'person', how = 'left')
sets = sets.merge(frecuencia_visitas, on = 'person', how = 'left')
sets.head()

In [ ]:
del frecuencia_compras
del frecuencia_vistas
del frecuencia_otros
del frecuencia_visitas
sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].fillna(value=0) 

In [ ]:
# Rellenamos compras_prod  vistas_prod  otros y visitas_sitio con 0
sets['compras_prod'] = sets['compras_prod'].fillna(value=0) 
sets['vistas_prod'] = sets['vistas_prod'].fillna(value=0) 
sets['otros'] = sets['otros'].fillna(value=0) 
sets['visitas_sitio'] = sets['visitas_sitio'].fillna(value=0) 

In [ ]:
sets.count()

In [ ]:
#  Nos quedamos con cant_eventos_x (ya que son iguales con cant_eventos_y) convertimos a int y Rellenamos con 0
sets['cant_eventos'] = sets['cant_eventos_x']
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')
sets.drop('cant_eventos_x',axis=1, inplace=True)
sets.drop('cant_eventos_y',axis=1, inplace=True)
sets.head()

In [ ]:
# Rellenamos model con 0 para luego hacer un split entre marca y modelo
sets['model'] = sets['model'].fillna(value='0') 

In [ ]:
# Rellenamos los demas con ''
sets = sets.fillna(value='') 

In [ ]:
sets.count()

In [ ]:
sets.person.describe()

## Categorizacion de los features

In [ ]:
# Rellenamos cantidad de eventos y convertimos a int
sets['cant_eventos'] = sets['cant_eventos'].fillna(value=0) 
sets['cant_eventos'] = sets['cant_eventos'].astype('int')

sets['cant_compras'] = sets['cant_compras'].fillna(value=0) 
sets['cant_compras'] = sets['cant_compras'].astype('int')
sets['cant_vistas'] = sets['cant_vistas'].fillna(value=0) 
sets['cant_vistas'] = sets['cant_vistas'].astype('int')
sets['cant_otros'] = sets['cant_otros'].fillna(value=0) 
sets['cant_otros'] = sets['cant_otros'].astype('int')
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].fillna(value=0) 
sets['cant_visitas_sitio'] = sets['cant_visitas_sitio'].astype('int')

In [ ]:
# Partimos el feature model en marca y modelo
sets['model'] = sets['model'].apply(lambda x: x.split(' '))
sets['marca'] = sets['model'].apply(lambda x: "apple" if str(x[0]) == "iPhone" else str(x[0]))
sets['modelo'] = sets['model'].apply(lambda x: ' '.join(x[1:]) if len(x)>1 else "0" )
sets.drop('model', axis=1, inplace=True)
sets.head()

In [ ]:
# creo una nueva columna person_int para entrenar transformando el valor alfanumerico de person a numerico
sets['person_int'] = labelencoder.fit_transform(sets['person'])
sets.head()

In [ ]:
# convertimos a integer el valor flotante de compras_prod
sets['compras_prod'] = sets['compras_prod'].astype('int')
sets.head()

In [ ]:
# convertimos a integer el valor flotante de vistas_prod
sets['vistas_prod'] = sets['vistas_prod'].astype('int')
sets.head()

In [ ]:
# convertimos a integer el valor flotante de otros
sets['otros'] = sets['otros'].astype('int')
sets.head()

In [ ]:
# convertimos a integer el valor flotante de visitas_sitio
sets['visitas_sitio'] = sets['visitas_sitio'].astype('int')
sets.head()

In [ ]:
# convertimos a string el valor flotante de sku
sets['sku'] = sets['sku'].astype('str')
sets.head()

In [31]:
#Categorizamos los valores de los features con label encoder
sets['sku'] = labelencoder.fit_transform(sets['sku'])
sets['marca'] = labelencoder.fit_transform(sets['marca'])
sets['modelo'] = labelencoder.fit_transform(sets['modelo'])
sets['condition'] = labelencoder.fit_transform(sets['condition'])
sets['storage'] = labelencoder.fit_transform(sets['storage'])
sets['color'] = labelencoder.fit_transform(sets['color'])
sets['new_vs_returning'] = labelencoder.fit_transform(sets['new_vs_returning'])
sets['region'] = labelencoder.fit_transform(sets['region'])
sets['country'] = labelencoder.fit_transform(sets['country'])
sets['device_type'] = labelencoder.fit_transform(sets['device_type'])
sets['screen_resolution'] = labelencoder.fit_transform(sets['screen_resolution'])
sets['operating_system_version'] = labelencoder.fit_transform(sets['operating_system_version'])
sets['browser_version'] = labelencoder.fit_transform(sets['browser_version'])
sets['url'] = labelencoder.fit_transform(sets['url'])
sets['skus'] = labelencoder.fit_transform(sets['skus'])
sets['search_term'] = labelencoder.fit_transform(sets['search_term'])
sets['staticpage'] = labelencoder.fit_transform(sets['staticpage'])
sets['campaign_source'] = labelencoder.fit_transform(sets['campaign_source'])
sets['search_engine'] = labelencoder.fit_transform(sets['search_engine'])
sets['channel'] = labelencoder.fit_transform(sets['channel'])
sets.head()

KeyError: 'marca'

In [ ]:
#Categorizamos la fecha del visto del producto en dia mes y dia de la semana  (ya que el anio es 2018)
sets['dia_view_prod'] = sets['fecha_x'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_view_prod'] = sets['fecha_x'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_view_prod'] = sets['fecha_x'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_x',1)

In [ ]:
#Categorizamos la fecha de visita del usuario en dia mes y dia de la semana (ya que el anio es 2018)
sets['dia_visit_user'] = sets['fecha_y'].apply(lambda x: x.day if type(x) != str else 0).astype('int')
sets['mes_visit_user'] = sets['fecha_y'].apply(lambda x: x.month if type(x) != str else 0).astype('int')
sets['dia_sem_visit_user'] = sets['fecha_y'].apply(lambda x: x.dayofweek if type(x) != str else 0).astype('int')
sets = sets.drop('fecha_y',1)

In [ ]:
sets.cant_eventos.describe()

In [ ]:
prediction_set = pd.read_csv('data/trocafone_kaggle_test.csv', low_memory = False)
training_set = pd.read_csv('data/labels_training_set.csv', low_memory=False)

print('PREDICCION ' , prediction_set.person.describe())
print('ENTRENAMIENTO ', training_set.person.describe())
print('ENTRENAMIENTO 1s ', training_set[training_set['label'] == 1].shape)
print('ENTRENAMIENTO 0s ', training_set[training_set['label'] == 0].shape)

prediction_set = prediction_set.merge(sets, on = 'person', how = 'inner')

training_set = training_set.merge(sets, on = 'person', how = 'inner')

In [ ]:
training_set.frecuencia.sort_values(ascending=False)

In [ ]:
prediction_set.person.describe()

In [ ]:
training_set[training_set['label'] == 0].shape

In [ ]:
prediction_set.shape

In [ ]:
training_set[training_set['label'] == 0].shape

In [ ]:
# Guardar Set

In [ ]:
training_set.to_csv('data/set_prediccion_21.csv', encoding='utf-8', index=False)

In [ ]:
prediction_set.to_csv('data/set_entrenamiento_21.csv', encoding='utf-8', index=False)